In [ ]:
import openai
import yaml
from src import utils
import time
from pprint import pprint
import os
import argparse
from tqdm import tqdm
from src.prompts import get_CoT_prompt, get_3p_prompt

In [ ]:
# PROMPT = {
#     '3p' : get_3p_prompt,
#     'cot' : get_CoT_prompt,
# }

In [ ]:
api = 'openai'
api_key_file = '/userhomes/philhoon/self-debate/api.yaml'

In [ ]:
# model_name="gpt-3.5-turbo-0301",
model_name = "gpt-3.5-turbo-0613"

max_token=2048
temperature=0.0
prompt_type='cot'

In [ ]:
data_name='gsm8k'
input_file = '/projects/self-debate/data/gsm8k/test.jsonl'
output_path = '/userhomes/philhoon/self-debate/result'

In [ ]:
# openai key
key_dict = utils.get_keys(api_key_file)
openai.api_key = key_dict[api]

In [ ]:
file_type = input_file.split('/')[-1].split('.')[-2]
output_file_name = f'{api}-{model_name}-{max_token}-{temperature}-KOR-{data_name}-{file_type}.jsonl'
os.makedirs(output_path, exist_ok=True)
output_file = output_path + '/' + output_file_name
print(f'output_file : {output_file}')

In [ ]:
# openai-gpt-3.5-turbo-0301-2048-0.0-gsm8k-test.jsonl -> openai-gpt-3.5-turbo-0301-3p-2048-0.0-gsm8k-test.jsonl
# openai-gpt-3.5-turbo-0301-gsm8k-test.jsonl -> openai-gpt-3.5-turbo-0301-3p-inf-1.0-gsm8k-test.jsonl
# openai-gpt-3.5-turbo-0301-2048-0.0-gsm8k-test.jsonl -> openai-gpt-3.5-turbo-0301-3p-2048-0.0-gsm8k-test.jsonl

In [ ]:
# output_file = output_path + '/' + 'gsm8k-korean.jsonl'
# print(output_file)

In [ ]:
# Jsonlines writier
jsonl_writer = utils.JSONLWriter(output_file)

In [ ]:
data = utils.read_json(input_file)

In [ ]:
def get_message(prompt):
    """
    Message for chat api
    """
    message = {
        "role" : "user",
        "content" : prompt
        }
    return [message]

In [ ]:
def generate_answer(message, model_name, max_token, temperature):
    """
    Generate response
    """
    try:
        completion = openai.ChatCompletion.create(
            model=model_name,
            messages=message,
            temperature=temperature,
            max_tokens=max_token,
            n=1)
    except:
        print("retrying due to an error......")
        time.sleep(20)
        return generate_answer(message, model_name, max_token, temperature)

    return completion

In [ ]:
def get_trans_msg(context):
    """
    Message for chat api
    """

    msg = [
        {
            "role" : "system",
            "content" : "Translate given setences into Korean."
        },
        {
            "role" : "user",
            "content" : context
        }
    ]
    
    return msg

In [ ]:
model_name

In [ ]:
max_token

In [ ]:
temperature

In [ ]:
# cnt = 0
for instance in tqdm(data):
    query = instance['question']
    ans = instance['answer']
    answer_s = utils.extract_answer(instance['answer'])
        
    query_msg = get_trans_msg(query)
    ans_msg = get_trans_msg(ans)
    
    
    query_completion = generate_answer(query_msg, model_name, max_token, temperature)
    q_response = query_completion["choices"][0]["message"]["content"]
    
    ans_completion = generate_answer(ans_msg, model_name, max_token, temperature)
    ans_response = ans_completion["choices"][0]["message"]["content"]

    instance['kor_question'] = q_response
    instance['kor_answer'] = ans_response
    instance['answer_s'] = answer_s
    
    pprint(instance)
#     jsonl_writer.write_json_line(instance)
#     cnt += 1
#     if cnt == 10:
#         break

In [ ]:
# cnt = 0
# for instance in data:
#     question = instance['question']
#     answer_s = utils.extract_answer(instance['answer'])
#     prompt = get_CoT_prompt(question)
#     prompt = get_prompt(question)
#     print(f'prompt : \n {prompt}')
#     message = get_message(prompt)
    
#     continue
#     completion = generate_answer(message, model_name)
    
# #     completion = openai.ChatCompletion.create(
# #                   model=model_name,
# #                   messages=message,
# #                   temperature=0,Pathways Language Model (PaLM): Scaling to 540 Billion Parameters for Breakthrough Performance
# #                   max_tokens=4096,
# #                   n=1)
    
    
#     response = completion["choices"][0]["message"]["content"]
# #     instance['response'] = response
# #     instance['answer_specific'] = answer_s
# #     jsonl_writer.write_json_line(instance)
#     print(f'respones : \n {response}')
#     cnt += 1
#     if cnt == 10:
#         break

In [ ]:
pprint(instance)